In [ ]:
import pandas as pd
import numpy as np
import re

import spacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

from sklearn.preprocessing import MinMaxScaler, StandardScaler
import torch.nn as nn
from sklearn.preprocessing import LabelEncoder
import joblib

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


## Загрузка данных

In [ ]:
df_main = pd.read_csv("payments_main.tsv", sep='\t', names=["id", "date", "amount", "text"])
df_main = df_main.drop(columns=["date"])
df_main.head(10)

,id,amount,text
0,1,40500.00,За тур.поездку по договору №001 от 27.01.2023г
1,2,"32600,00",За оказание услуг по договору №53Б-02746 от 23...
2,3,4710-00,Оплата штрафа
3,4,30900-00,Лечение по договору №Д-00359/24 от 08.03.2025
4,5,13200.00,Оплата основного долга за период с 16.12.2024г...
5,6,4210.00,Оплата за Бульон Роллтон Домашний куриный 90г ...
6,7,4240-00,Комиссионное вознаграждение за валютный перевод.
7,8,4630.00,государственная пошлина
8,9,8000.00,Лечение по договору №Д00184/63 от 27.12.2023 г.
9,10,"1310000,00","Оплата по счету 0187,0188,0189 от 02.01.2024 (..."


In [ ]:
df_main.shape

(25000, 3)

## Предобработка данных

#### Очищаем amount и text, приводим к единому формату

In [ ]:
def parse_amount(amount):
    if not isinstance(amount, str):
        return None

    amount = amount.replace(" ", "")
    amount = amount.replace("-", ".")


    if ',' in amount and '.' in amount:
        # Mixed format (e.g., 14.000,00 or 14,000.00)
        if amount.index(',') > amount.index('.'):
            # Format like 14.000,00 (dot as thousand separator, comma as decimal separator)
            amount = amount.replace('.', '').replace(',', '.')
        else:
            # Format like 14,000.00 (comma as thousand separator, dot as decimal separator)
            amount = amount.replace(',', '')
    elif ',' in amount:
        if amount.index(',') < len(amount)-3:
            amount = amount.replace(',', '')
        else:
            amount = amount.replace(',', '.')
    elif '.' in amount:
        # Format like 14.000 or 14000.00 (dot as thousand separator or decimal point)
        amount = re.sub(r'\.(?=\d{3}(?!\d))', '', amount)  # Remove dots used as thousand separator

    # Convert to float
    try:
        return float(amount)
    except ValueError:
        return None

In [ ]:
def clean_meta_info(text):
    # Преобразовать в нижний регистр
    text = text.lower()

    # Удалить номера договоров: "№452", "№E01368" и подобное
    text = re.sub(r"№\S+", " ", text)
    # Удалить даты в форматах: "17.03.2024", "17-03-24", "17/03/2024", "17.03.2024г", "01/01/2024г"
    text = re.sub(r"\d{1,2}[-./]\d{1,2}[-./]\d{2,4}\s?г?\.?", " ", text)
    # Удалить год в формате: "2024г"
    text = re.sub(r"\b\d{4}\s?г\.?\b", " ", text)
    # Удалить названия месяцев
    text = re.sub(r'\b(?:января|февраля|марта|апреля|мая|июня|июля|августа|сентября|октября|ноября|декабря)\b', ' ', text)
    # Удалить суммы: "100 000.50", "2400000,00", "2400000.00", "100000-50"
    text = re.sub(r"\b\d{1,3}([ .,-]?\d{3})*(\.\d+|,\d+|-?\d+)?\b", " ", text)
    # Удалить символы валют
    text = re.sub(r'\b(?:₽|доллар(?:ов|а)?|USD|RUB|руб(?:.|ль|ля|лей)?)\b', ' ', text, flags=re.IGNORECASE)

    # Удалить служебные слова без информации: "на сумму", "от", "в т.ч."
    text = re.sub(r"\b(?:сумма|на сумму|от|и т\.д\.|в т\.ч\.?|в том числе|г\.)\b", " ", text, flags=re.IGNORECASE)


    # Удалить отдельно стоящие дефисы
    text = re.sub(r'-+', '-', text)
    # Удалить процентные знаки
    text = re.sub(r'%', ' ', text)
    # Удалить скобки
    text = re.sub(r'[()]', ' ', text)
    # Удалить лишние запятые
    text = re.sub(r"\s{2,}", " ", text).strip()

    text = text.replace(".", " ")
    text = text.replace("/", " ")

    # Удалить отдельно стоящие дефисы
    text = re.sub(r'(?<!\w)-(?!\w)|-+(?=\s|$)', '', text)
    # Удалить слова короче 3 символов
    text = re.sub(r'\s+\w{1,2}\s+', ' ', text)

    # Удаление лишних пробелов
    text = re.sub(r'\s+', ' ', text)

    return text

In [ ]:
try:
    df_main['amount'] = df_main['amount'].apply(parse_amount)
    df_main['cleaned_text'] = df_main['text'].apply(clean_meta_info)
except:
    pass
df_main.head(20)

,id,amount,text,cleaned_text
0,1,40500.0,За тур.поездку по договору №001 от 27.01.2023г,за тур поездку договору
1,2,32600.0,За оказание услуг по договору №53Б-02746 от 23...,за оказание услуг договору
2,3,4710.0,Оплата штрафа,оплата штрафа
3,4,30900.0,Лечение по договору №Д-00359/24 от 08.03.2025,лечение договору
4,5,13200.0,Оплата основного долга за период с 16.12.2024г...,оплата основного долга период по договору оао ...
5,6,4210.0,Оплата за Бульон Роллтон Домашний куриный 90г ...,оплата бульон роллтон домашний куриный 90г счету
6,7,4240.0,Комиссионное вознаграждение за валютный перевод.,комиссионное вознаграждение валютный перевод
7,8,4630.0,государственная пошлина,государственная пошлина
8,9,8000.0,Лечение по договору №Д00184/63 от 27.12.2023 г.,лечение договору
9,10,1310000.0,"Оплата по счету 0187,0188,0189 от 02.01.2024 (...","оплата счету , , рамках договора финансовой ар..."


#### Применяем стемминг и лемматизацию

In [ ]:
!python -m spacy download ru_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 62.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
nlp = spacy.load("ru_core_news_sm")

def preprocess(text, stop_words, punctuation_marks):
    tokens = word_tokenize(text.lower())
    preprocessed_text = []

    for token in tokens:
        if len(token) < 3:
            continue
        if token[0] == '-' or token[-1] == '-':
            continue
        if token not in punctuation_marks:
            doc = nlp(token)
            lemma = doc[0].lemma_
            if lemma not in stop_words:
                preprocessed_text.append(lemma)
    return preprocessed_text

In [ ]:
punctuation_marks = ['!', ',', '(', ')', ':', '-', '?', '.', '..', '...']
stop_words = stopwords.words("russian")

In [ ]:
df_main['preprocessed_text'] = df_main.apply(lambda row: preprocess(row['cleaned_text'], stop_words, punctuation_marks), axis=1)
df_main.head(10)

,id,amount,text,cleaned_text,preprocessed_text
0,1,40500.0,За тур.поездку по договору №001 от 27.01.2023г,за тур поездку договору,"[тур, поездка, договор]"
1,2,32600.0,За оказание услуг по договору №53Б-02746 от 23...,за оказание услуг договору,"[оказание, услуга, договор]"
2,3,4710.0,Оплата штрафа,оплата штрафа,"[оплата, штраф]"
3,4,30900.0,Лечение по договору №Д-00359/24 от 08.03.2025,лечение договору,"[лечение, договор]"
4,5,13200.0,Оплата основного долга за период с 16.12.2024г...,оплата основного долга период по договору оао ...,"[оплата, основный, долг, период, договор, оао,..."
5,6,4210.0,Оплата за Бульон Роллтон Домашний куриный 90г ...,оплата бульон роллтон домашний куриный 90г счету,"[оплата, бульон, роллтон, домашний, куриный, 9..."
6,7,4240.0,Комиссионное вознаграждение за валютный перевод.,комиссионное вознаграждение валютный перевод,"[комиссионный, вознаграждение, валютный, перевод]"
7,8,4630.0,государственная пошлина,государственная пошлина,"[государственный, пошлина]"
8,9,8000.0,Лечение по договору №Д00184/63 от 27.12.2023 г.,лечение договору,"[лечение, договор]"
9,10,1310000.0,"Оплата по счету 0187,0188,0189 от 02.01.2024 (...","оплата счету , , рамках договора финансовой ар...","[оплата, счёт, рамка, договор, финансовый, аре..."


In [ ]:
preprocessed_df_main = df_main[['id', 'amount', 'preprocessed_text']]
preprocessed_df_main.head(20)

,id,amount,preprocessed_text
0,1,40500.0,"[тур, поездка, договор]"
1,2,32600.0,"[оказание, услуга, договор]"
2,3,4710.0,"[оплата, штраф]"
3,4,30900.0,"[лечение, договор]"
4,5,13200.0,"[оплата, основный, долг, период, договор, оао,..."
5,6,4210.0,"[оплата, бульон, роллтон, домашний, куриный, 9..."
6,7,4240.0,"[комиссионный, вознаграждение, валютный, перевод]"
7,8,4630.0,"[государственный, пошлина]"
8,9,8000.0,"[лечение, договор]"
9,10,1310000.0,"[оплата, счёт, рамка, договор, финансовый, аре..."


In [ ]:
texts = [" ".join(text) for text in preprocessed_df_main['preprocessed_text']]
preprocessed_df_main['preprocessed_text'] = texts

preprocessed_df_main.head()

<ipython-input-35-716614bc8209>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_df_main['preprocessed_text'] = texts


,id,amount,preprocessed_text
0,1,40500.0,тур поездка договор
1,2,32600.0,оказание услуга договор
2,3,4710.0,оплата штраф
3,4,30900.0,лечение договор
4,5,13200.0,оплата основный долг период договор оао второй...


In [ ]:
preprocessed_df_main.to_csv('preprocessed_main.tsv', sep='\t')

## Загрузка модели и прогон данных

In [ ]:
preprocessed_df_main = pd.read_csv("preprocessed_training.tsv", sep='\t')
preprocessed_df_main = preprocessed_df_main.drop(columns=["Unnamed: 0"])
preprocessed_df_main.head()

,id,amount,preprocessed_text,category
0,1,15300.0,участие конференция майкоп договор,SERVICE
1,2,40200.0,оказание услуга договор,SERVICE
2,3,1440.0,оплата порошок стиральный ariel color automat ...,NON_FOOD_GOODS
3,4,240000000.0,возврат денежный средство договор заём ндс,LOAN
4,5,1360000.0,оплата дог соглый оплата сброс загрязнять веще...,NOT_CLASSIFIED


### 1. Создание эмбеддингов текстового столбца

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1. Определяем Dataset
class TextDataset(Dataset):
    def __init__(self, texts):
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return self.texts[idx]

# 2. Инициализация токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained('cointegrated/rubert-tiny')
model_rubert = AutoModel.from_pretrained('cointegrated/rubert-tiny').to(device)

texts = preprocessed_df_main['preprocessed_text']

# 3. Создаём Dataset и DataLoader
text_dataset = TextDataset(texts)
batch_size = 512
data_loader = DataLoader(text_dataset, batch_size=batch_size, shuffle=False)

# 4. Обработка данных батчами
embeddings = []

model_rubert.eval()
with torch.no_grad():
    for batch in tqdm(data_loader, desc="Processing batches", total=len(data_loader), ncols=100):
        encoded_inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt")

        input_ids = encoded_inputs['input_ids'].to(device)
        attention_mask = encoded_inputs['attention_mask'].to(device)

        outputs = model_rubert(input_ids=input_ids, attention_mask=attention_mask)

        cls_embeddings = outputs.last_hidden_state[:, 0, :]
        embeddings.append(cls_embeddings)

final_embeddings = torch.cat(embeddings, dim=0)

print(final_embeddings.shape)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Processing batches: 100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 15.98it/s]

torch.Size([500, 312])


In [ ]:
final_embeddings_list = [emb.cpu().numpy() for emb in final_embeddings]

preprocessed_df_main['text_embed'] = final_embeddings_list

preprocessed_df_main[['preprocessed_text', 'text_embed']].head()

,preprocessed_text,text_embed
0,участие конференция майкоп договор,"[-0.3839216, 0.029149303, 0.041437216, 0.09084..."
1,оказание услуга договор,"[-0.73675907, -0.35754037, -0.26379448, -0.297..."
2,оплата порошок стиральный ariel color automat ...,"[-0.012440931, -0.09139417, -0.20764157, -0.69..."
3,возврат денежный средство договор заём ндс,"[-1.0594479, -0.52556884, -0.5544399, -0.36923..."
4,оплата дог соглый оплата сброс загрязнять веще...,"[-0.2358977, -0.50058454, -0.5359852, -0.76608..."


### 2. Нормализация amount и объединение данных

In [ ]:
scaler = MinMaxScaler()

preprocessed_df_main['amount_normalized'] = scaler.fit_transform(preprocessed_df_main[['amount']])
preprocessed_df_main[['amount', 'amount_normalized']].head()

,amount,amount_normalized
0,15300.0,0.000032
1,40200.0,0.000087
2,1440.0,0.000002
3,240000000.0,0.525164
4,1360000.0,0.002975


In [ ]:
hidden_size = 768

class AmountProcessor(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(AmountProcessor, self).__init__()
        self.linear = nn.Linear(input_dim, output_dim)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.linear(x)
        x = self.activation(x)
        return x

amount_processor = AmountProcessor(input_dim=1, output_dim=hidden_size)

text_embeddings_array = np.stack(preprocessed_df_main['text_embed'].values)  # [num_samples, hidden_size]
amount_normalized_array = preprocessed_df_main['amount_normalized'].values.reshape(-1, 1)  # [num_samples, 1]

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
amount_processor = amount_processor.to(device)

text_embeddings_tensor = torch.tensor(text_embeddings_array, dtype=torch.float32).to(device)
amount_normalized_tensor = torch.tensor(amount_normalized_array, dtype=torch.float32).to(device)

processed_amount = amount_processor(amount_normalized_tensor)  # [num_samples, hidden_size]

combined_features = torch.cat((text_embeddings_tensor, processed_amount), dim=1)  # [num_samples, 2 * hidden_size]

print("Размер объединённых признаков:", combined_features.shape)

Размер объединённых признаков: torch.Size([500, 1080])


In [ ]:
combined_features_cpu = combined_features.cpu().detach().numpy()
combined_features_list = [feature.tolist() for feature in combined_features_cpu]

preprocessed_df_main['combined_features'] = combined_features_list
preprocessed_df_main[['preprocessed_text', 'text_embed', 'amount_normalized', 'combined_features']].head(1)

,preprocessed_text,text_embed,amount_normalized,combined_features
0,участие конференция майкоп договор,"[-0.3839216, 0.029149303, 0.041437216, 0.09084...",0.000032,"[-0.3839215934276581, 0.02914930321276188, 0.0..."


In [ ]:
preprocessed_df_main.columns

Index(['id', 'amount', 'preprocessed_text', 'category', 'text_embed',
       'amount_normalized', 'combined_features'],
      dtype='object')

In [ ]:
preprocessed_df_main = preprocessed_df_main[['id', 'amount', 'preprocessed_text', 'category', 'combined_features']]

# preprocessed_df_main.to_csv("embedded_main.tsv", sep='\t', index=False)

### 3. Загрузка обученной модели и получение предсказаний

In [ ]:
categories = ['BANK_SERVICE', 'FOOD_GOODS', 'LEASING', 'LOAN', 'NON_FOOD_GOODS', 'NOT_CLASSIFIED', 'REALE_STATE', 'SERVICE', 'TAX']
num_classes = len(categories)

In [ ]:
class ClassificationModel(nn.Module):
    def __init__(self, input_dim, hidden_dims, num_classes, dropout_prob=0.3):
        super(ClassificationModel, self).__init__()
        layers = []

        for hidden_dim in hidden_dims:
            layers.append(nn.Linear(input_dim, hidden_dim))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout_prob))
            input_dim = hidden_dim

        layers.append(nn.Linear(hidden_dims[-1], num_classes))

        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x)

In [ ]:
X_tensor = torch.tensor(preprocessed_df_main['combined_features'], dtype=torch.float32).to(device)

input_dim = X_tensor.shape[1]
hidden_dims = [1024, 512, 256]


model = ClassificationModel(input_dim=input_dim, hidden_dims=hidden_dims, num_classes=num_classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model.load_state_dict(torch.load('model.pth', map_location=torch.device('cpu')))
model.to(device)

<ipython-input-42-3b0bde148c1b>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('model.pth', map_location=torch.device('cpu')))


ClassificationModel(
  (net): Sequential(
    (0): Linear(in_features=1080, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=512, out_features=256, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.3, inplace=False)
    (9): Linear(in_features=256, out_features=9, bias=True)
  )
)

#### Предсказание

In [ ]:
model.eval()

with torch.no_grad():
    outputs = model(X_tensor)

_, predictions = outputs.max(1)

label_encoder = joblib.load('label_encoder.pkl')
# abel_encoder = LabelEncoder()
# label_encoder.fit(categories)

predicted_labels = label_encoder.inverse_transform(predictions.cpu().numpy())

print(set(predicted_labels))

{'LEASING', 'LOAN', 'BANK_SERVICE', 'NOT_CLASSIFIED', 'NON_FOOD_GOODS', 'FOOD_GOODS', 'REALE_STATE', 'SERVICE', 'TAX'}


In [ ]:
preprocessed_df_main['predicted_label'] = predicted_labels

output_df = preprocessed_df_main[['id', 'preprocessed_text', 'category', 'predicted_label']]
output_df.head(100)

,id,preprocessed_text,category,predicted_label
0,1,участие конференция майкоп договор,SERVICE,SERVICE
1,2,оказание услуга договор,SERVICE,SERVICE
2,3,оплата порошок стиральный ariel color automat ...,NON_FOOD_GOODS,NON_FOOD_GOODS
3,4,возврат денежный средство договор заём ндс,LOAN,LOAN
4,5,оплата дог соглый оплата сброс загрязнять веще...,NOT_CLASSIFIED,NOT_CLASSIFIED
...,...,...,...,...
95,96,оплата бумажный полотенце zewa лист рулон счёт,NON_FOOD_GOODS,FOOD_GOODS
96,97,возврат денежный средство договор заём ндс,LOAN,LOAN
97,98,транспортировка перевозка автомобиль старый ос...,SERVICE,SERVICE
98,99,оплата рукав запекание econta счёт,NON_FOOD_GOODS,NON_FOOD_GOODS


In [ ]:
count = 0
for i in range(len(output_df)):
    if output_df.at[i, 'predicted_label'] == output_df.at[i, 'category']:
        count += 1

print("Accuracy: ", f'{count / len(output_df):.2f}')

Accuracy:  0.85
